전이학습(transfer learning) : 방대한 자료를 통해 미리 학습한 가중치(weight) 값을 가져와 프로젝트에 활용하는 것

##### 소규모 데이터셋으로 만드는 강력한 학습 모델

지도 학습(supervised learning) : 폐암환자 생존율, 피마인디언 당뇨, MNIST 등 값을 지정하고 학습

비지도 학습(unsupervised learning) : GAN, AutoEncoder 등 주어진 데이터의 특성을 찾도록 학습


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, initializers, regularizers, metrics

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


np.random.seed(3)
tf.random.set_seed(3)

"""
- rescale : 주어진 이미지 크기를 바꿔줌. 1./255 -> 0~255의 RGB를 0~1 사이의 값으로 바꾸어줌, 학습이 빨라짐
- horizontal_flip, vertical_flip : 주어진 이미지를 수평 또는 수직으로 뒤집음
- zoom_range : 정해진 범위 내에서 축소, 확대
- width_shift, height_shift : 정해진 범위 안에서 수평 또는 수직으로 랜덤하게 평행 이동시킴
- rotation_range : 정해진 각도만큼 이미지를 회전시킴
- shear_range : 좌표 하나를 고정시키고 다른 몇개의 좌표를 이동시키는 변환
- fill_mode : 이미지를 축소 또는 회전하거나 이동할때 생기는 빈 공간을 어떻게 채울지 결정. nearest 옵션을 선택하면 가장 비슷한 색으로 채워짐
"""
traln_datagen = ImageDataGenrator(rescale = 1. / 255,
                                  horizontal_flip = True,
                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1,
                                  fill_mode = 'nearest')

train_generator = train_datagen.flow_from_directory(
    './train',                     # train 데이터셋이 저장된 폴더
    target_size = (150, 150),
    batch_size = 5,
    class_mode = 'binary'   
)

# 테스트 셋은 이미지 부풀리기 과정을 진행하지 않음
test_datagen = ImageDataGenerator(rescale = 1. / 255)

test_generator = test_datagen.flow_from_directory(
    './test',
    target_size = (150, 150),
    batch_size = 5,
    class_mode = 'binary'
)

# 앞서 배운 CNN 모델을 만들어 적용하기
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

# 모델 컴파일
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = optimizers.Adam(learning_rate = 2e-4),
              metrics = ['accuracy'])

# 모델 실행
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 100,
    epochs = 20,
    validation_data = test_generator,
    validation_steps = 4
)

# 결과를 그래프로 표현하는 부분
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
y_vloss = history.history['val_loss']
y_loss = history.history['loss']
x_len = np.arange(len(y_loss))

plt.plot(x_len, acc, marker = '.', c = 'red', label = 'Trainset_acc')
plt.plot(x_len, val_acc, marker = '.', c = 'lightcoral', label = 'Testset_acc')

plt.plot(x_len, y_vloss, marker = '.', c = 'conflowerblue', label = 'Testset_loss')
plt.plot(x_len, y_loss, marker = '.', c = 'blue', label = 'Trainset_loss')

plt.legend(loc = 'upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss/acc')
plt.show()

##### 전이 학습으로 모델 성능 극대화하기

VGGNet : 옥스포드의 연구팀 VGG에서 개발된 모델(층에 따라 VGG16, VGG19 등으로 불림)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input, models, layers, optimizers, metrics
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(3)
tf.compat.v1.set_random_seed(3)

train_datagen = ImageDataGenrator(rescale = 1. /255,
                                  horizontal_flip = True,
                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1,
                                  fill_mode = 'nearest')

train_generator = train_datagent.flow_from_directory(
    'train',
    target_size = (150, 150),
    batch_size = 5,
    class_mode = 'binary'
)

test_datagen = ImageDataGenerator(rescale = 1. / 255,
                                  horizontal_flip = True,
                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1,
                                  fill_mode = 'nearest')

test_generator = test_datagen.flow_from_directory(
    'test',
    target_size = (150, 150),
    batch_size = 5,
    class_mode = 'binary'
)

transfer_model = VGG16(weights = 'imagenet',
                       include_top = False,
                       input_shape = (150, 150, 3))
transfer_model.trainable = False
transfer_model.summary()

finetune_model = models.Sequential()
finetune_model.add(transfer_model)
finetune_model.add(Flatten())
finetune_model.add(Dense(64, activation = 'relu'))
finetune_model.add(Dense(2, activation = 'softmax'))
finetune_model.summary()

finetune_model.compile(loss = 'sparse_categorical_crossentropy',
                       optimizer = optimizers.Adam(learning_rate = 2e-4),
                       metrics = ['accuracy'])

history = finetune_model.fit_generator(train_generator,
                                       steps_per_epoch = 100,
                                       epochs = 20,
                                       validation_data = test_generator,
                                       validation_steps = 4)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))

plt.plot(x_len, acc, marker = '.', c = 'red', label = 'Trainset_acc')
plt.plot(x_len, val_acc, marker = '.', c = 'lightcoral', label = 'Testset_acc')

plt.plot(x_len, y_vloss, marker = '.', c = 'cornflowerblue', label = 'Testset_loss')
plt.plot(x_len, y_loss, marker = '.', c = 'blue', label = 'Trainset_loss')

plt.legend('upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss/acc')

plt.show()